<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/Fake_News_Classification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
import pandas as pd
import os

In [4]:
!unzip /content/drive/MyDrive/train_LSTM.zip


Archive:  /content/drive/MyDrive/train_LSTM.zip
  inflating: train_LSTM.csv          


In [5]:
df=pd.read_csv('/content/train_LSTM.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
df.reset_index(drop=True,inplace=True)

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
X=df.drop('label',axis=1)
y=df['label']

In [15]:
X.shape

(18285, 4)

In [37]:
X.drop('id',axis=1,inplace=True)

In [3]:
import tensorflow as tf


In [4]:
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
messages1=X.copy()

## DATA CLEANING


In [27]:
def tolower(text):
  return text.lower()

In [39]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(tolower)

In [41]:
messages1.head()

,title,author,text
0,house dem aide: we didn’t even see comey’s let...,darrell lucus,house dem aide: we didn’t even see comey’s let...
1,"flynn: hillary clinton, big woman on campus - ...",daniel j. flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnews.com,"why the truth might get you fired october 29, ..."
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [42]:
## REMOVING HTML TAGS
import re
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [44]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_html_tags)

In [45]:
## removing url
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [46]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_url)

In [48]:
## HANDLING PUNCTUATION
import string
def remove_punc(text):
  for i in string.punctuation:
    text=text.replace(i,'')
  return text


In [49]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_punc)

In [50]:
messages1.head()

,title,author,text
0,house dem aide we didn’t even see comey’s lett...,darrell lucus,house dem aide we didn’t even see comey’s lett...
1,flynn hillary clinton big woman on campus bre...,daniel j flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnewscom,why the truth might get you fired october 29 2...
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [51]:
## REMOVING STOPWORDS
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
def remove_stopwords(text):
  words=text.split()
  for i,word in enumerate(words):
    if word in stopwords.words('english'):
      words[i]=''
  return ' '.join(words)

In [53]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_stopwords)

In [55]:
messages1['title'][2]

'  truth might get  fired'

In [56]:
def remove_extra_spaces(sentence):

    cleaned_sentence = ' '.join(sentence.split()).strip()

    return cleaned_sentence

In [57]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_extra_spaces)

In [58]:
messages1['title'][2]

'truth might get fired'

In [59]:
messages1.to_csv('cleaned_data.csv')

In [1]:
!unzip /content/cleaned_data.zip

Archive:  /content/cleaned_data.zip
  inflating: cleaned_data.csv        


In [5]:
msg=pd.read_csv('/content/cleaned_data.csv')

In [6]:
msg=msg['title']

In [7]:
msg

0        house dem aide didn’t even see comey’s letter ...
1         flynn hillary clinton big woman campus breitbart
2                                    truth might get fired
3        15 civilians killed single us airstrike identi...
4        iranian woman jailed fictional unpublished sto...
                               ...                        
18280       rapper ti trump ’poster child white supremacy’
18281    nfl playoffs schedule matchups odds new york t...
18282    macy’s said receive takeover approach hudson’s...
18283          nato russia hold parallel exercises balkans
18284                                      keeps f35 alive
Name: title, Length: 18285, dtype: object

In [13]:
sentences=list(msg)

In [14]:
sentences

['house dem aide didn’t even see comey’s letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 '15 civilians killed single us airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathrooms exclusive video breitbart',
 'benoît hamon wins french socialist party’s presidential nomination new york times',
 'backchannel plan ukraine russia courtesy trump associates new york times',
 'obama’s organizing action partners soroslinked ‘indivisible’ disrupt trump’s agenda',
 'bbc comedy sketch real housewives isis causes outrage',
 'russian researchers discover secret nazi military base ‘treasure hunter’ arctic photos',
 'us officials see link trump russia',
 'yes paid government trolls social media blogs forums websites',
 'major league soccer argentines find home success new york times',
 'wells fargo chief abruptly 

In [18]:
sentences = [str(sentence) for sentence in sentences]

In [19]:
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 5000   #vocabulary size
onehot_repe = [one_hot(i,voc_size)for i in sentences]      #The one_hot function will encode words into integers, and the integers will range from 0 to voc_size - 1.
onehot_repe

[[2347, 4145, 2786, 4453, 3037, 2693, 2780, 4969, 1346, 1401, 1296],
 [2724, 1834, 1231, 1311, 238, 3962, 1227],
 [3191, 3647, 768, 1782],
 [1726, 469, 4984, 3986, 2480, 3602, 2918],
 [523, 238, 341, 3396, 1635, 128, 238, 3741, 1866, 1335],
 [3713,
  446,
  4026,
  1130,
  290,
  2322,
  3487,
  513,
  71,
  61,
  1946,
  2410,
  1474,
  4578,
  1227],
 [1232, 257, 3391, 2405, 4395, 3875, 3022, 3329, 478, 2592, 2573],
 [451, 2614, 1406, 1515, 809, 2322, 2892, 478, 2592, 2573],
 [938, 3201, 1953, 1306, 44, 2956, 4679, 2578, 4345],
 [3777, 4900, 1186, 3040, 1589, 2387, 3067, 3214],
 [3093, 281, 3080, 1251, 438, 1399, 2650, 2420, 832, 4953, 2337],
 [2480, 754, 2693, 701, 2322, 1515],
 [3852, 2841, 3181, 3847, 189, 1732, 3818, 4710, 515],
 [4369, 2292, 943, 433, 1300, 1846, 1497, 478, 2592, 2573],
 [2381, 396, 3256, 2881, 1225, 478, 2592, 2573],
 [3184, 2669, 3978, 3591, 3184, 2086, 3729, 2890, 3661, 4338, 1021],
 [1762, 2309, 1834],
 [4195, 3247, 3390, 2543, 2322, 2299, 767, 1227],
 [694,

In [20]:
onehot_repe[0]

[2347, 4145, 2786, 4453, 3037, 2693, 2780, 4969, 1346, 1401, 1296]

In [21]:
sentences[0]

'house dem aide didn’t even see comey’s letter jason chaffetz tweeted'

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 50
embedded_doc = pad_sequences(onehot_repe, maxlen = sent_length, padding = 'pre')

In [23]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 2347, 4145, 2786, 4453, 3037,
       2693, 2780, 4969, 1346, 1401, 1296], dtype=int32)

## CREATING MODEL

In [24]:
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310501 (1.18 MB)
Trainable params: 310501 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
